In [1]:
!yes | pip uninstall torchvison
!pip install -qU torchvision

yes: standard output: Broken pipe
yes: write error
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [29]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

## Data
### Getting the data



In [3]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [30]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-897173335964/sagemaker/DEMO-pytorch-mnist


## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [31]:
!pygmentize mnist.py

import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 32

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [32]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=2,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 25,
                        'backend': 'gloo'
                    })

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [33]:
estimator.fit({'training': inputs})

2022-08-20 18:25:09 Starting - Starting the training job...
2022-08-20 18:25:33 Starting - Preparing the instances for trainingProfilerReport-1661019908: InProgress
......
2022-08-20 18:26:35 Downloading - Downloading input data.........
2022-08-20 18:27:56 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-20 18:27:58,324 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-20 18:27:58,326 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-20 18:27:58,335 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-20 18:27:58,343 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-20 18:27:58,741 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-20 18:27:58,

Train Epoch: 1 [6400/30000 (21%)] Loss: 0.430215
Train Epoch: 1 [6400/30000 (21%)] Loss: 0.305484
Train Epoch: 1 [12800/30000 (43%)] Loss: 0.140919
Train Epoch: 1 [12800/30000 (43%)] Loss: 0.185905
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.189997
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.146270
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.025706
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.133261
Test set: Average loss: -11.8613, Accuracy: 9729/10000 (97%)
Test set: Average loss: -11.8613, Accuracy: 9729/10000 (97%)
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.108545
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.036331
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.080563
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.037911
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.174579
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.043166
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.016903
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.097241
Test set: Average loss: -13.9490, Accuracy: 9855/10000 (99%)
Test set: Average los

Train Epoch: 14 [19200/30000 (64%)] Loss: 0.003267
Train Epoch: 14 [19200/30000 (64%)] Loss: 0.019149
Train Epoch: 14 [25600/30000 (85%)] Loss: 0.000941
Train Epoch: 14 [25600/30000 (85%)] Loss: 0.002633
Test set: Average loss: -19.4220, Accuracy: 9938/10000 (99%)
Test set: Average loss: -19.4220, Accuracy: 9938/10000 (99%)
Train Epoch: 15 [6400/30000 (21%)] Loss: 0.037306
Train Epoch: 15 [6400/30000 (21%)] Loss: 0.005348
Train Epoch: 15 [12800/30000 (43%)] Loss: 0.061951
Train Epoch: 15 [12800/30000 (43%)] Loss: 0.000105
Train Epoch: 15 [19200/30000 (64%)] Loss: 0.000460
Train Epoch: 15 [19200/30000 (64%)] Loss: 0.009331
Train Epoch: 15 [25600/30000 (85%)] Loss: 0.019770
Train Epoch: 15 [25600/30000 (85%)] Loss: 0.004556
Test set: Average loss: -23.1815, Accuracy: 9941/10000 (99%)
Test set: Average loss: -23.1815, Accuracy: 9941/10000 (99%)
Train Epoch: 16 [6400/30000 (21%)] Loss: 0.028319
Train Epoch: 16 [6400/30000 (21%)] Loss: 0.000996
Train Epoch: 16 [12800/30000 (43%)] Loss: 0.00


2022-08-20 18:55:42 Uploading - Uploading generated training model
2022-08-20 18:55:42 Completed - Training job completed
Training seconds: 3480
Billable seconds: 3480


## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [35]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


---------!

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

In [36]:
!ls data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [56]:
from IPython.display import HTML
import numpy as np
HTML(open("input.html").read()) # it will read to "data" variable

In [77]:
image=np.array([data],dtype=np.float32)
response=predictor.predict(image)
prediction=response.argmax(axis=1)[0]
print(prediction)

(1, 1, 28, 28)
0


In [39]:
import gzip 
import numpy as np
import random
import os

data_dir = 'data/MNIST/raw'
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16) # randomly select some of the test images
mask = np.array(mask, dtype=np.int)
data = images[mask]

/tmp/ipykernel_28503/249326338.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.array(mask, dtype=np.int)


In [40]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

Raw prediction result:
[[-4.59496613e+02 -2.46716263e+02 -1.93678619e+02  4.61447662e+02
  -1.17260902e+02  9.78853088e+02 -2.15920506e+01 -1.89062576e+02
  -1.47251694e+02 -1.14774834e+02]
 [ 1.48833337e+03 -5.67534790e+02  1.74054749e+02 -4.69737518e+02
  -1.95035843e+02 -2.82980347e+02  1.79137192e+02 -2.76367310e+02
   1.11366447e+02  2.43404068e+02]
 [-7.40901413e+01 -1.72077698e+02  1.67234390e+02  7.83220215e+01
  -5.46099792e+02  9.44311829e+01  1.86217249e+03 -5.04577972e+02
   1.13772789e+02 -7.65186951e+02]
 [-7.40483826e+02 -2.27723713e+01  2.14171791e+01 -6.68054810e+02
   1.71603772e+03 -2.49430878e+02  9.83079605e+01 -2.18061142e+02
  -2.38755920e+02 -6.78035126e+01]
 [-2.83792171e+01 -2.88821320e+02  2.30950851e+02  9.81020737e+01
  -4.35517540e+01 -1.83035797e+02 -1.34090530e+02 -2.34209229e+02
   9.26288635e+02 -2.50381317e+01]
 [-2.76447723e+02 -3.62600006e+02  2.22685974e+02  1.60573071e+03
  -7.76958801e+02 -2.94470673e+01 -3.43540771e+02 -1.90411362e+02
   8.06890

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
# sagemaker_session.delete_endpoint(
#     endpoint_name = predictor.endpoint_name
# )